# Week 5 — Deployment on AWS SageMaker

**Learning Objectives (Week 5 – Deployment on AWS SageMaker)**  
- Package and deploy a model to a SageMaker endpoint.  
- Verify contracts: input schema, output schema, performance (latency/QPS).  
- Separate dev, pre-prod, prod deployments and promote safely.

## Exercises
1. Configure AWS credentials/role, region, S3 bucket.  
2. Upload model artifacts and create a SageMaker model.  
3. Deploy to a real-time endpoint; invoke and validate responses.  
4. (Optional) Blue/green or canary deployment strategy for updates.

## Peer Validation
- **Peer Review Checklist:**  
  - Endpoint created and reachable.  
  - Test invocations logged with sample payloads.  
  - Rollback plan articulated.

In [ ]:
# %pip install sagemaker boto3 --quiet
import os, json, boto3

# ---- Configuration (edit) ----
AWS_REGION = os.getenv("AWS_REGION","eu-west-1")
S3_BUCKET  = os.getenv("S3_BUCKET","your-s3-bucket")
ROLE_ARN   = os.getenv("SM_ROLE_ARN","arn:aws:iam::123456789012:role/SageMakerExecutionRole")

print("Edit AWS settings as needed. This is a scaffold for deployment code.")

In [ ]:
# Example scaffold using sagemaker SDK for a prebuilt sklearn container
# NOTE: This is illustrative and won't run here without AWS credentials.
from sagemaker.sklearn import SKLearnModel
from sagemaker import Session

sm_sess = Session()
model_artifact = "s3://{}/models/logreg/model.tar.gz".format(os.getenv("S3_BUCKET","your-s3-bucket"))

sk_model = SKLearnModel(
    model_data=model_artifact,
    role=os.getenv("SM_ROLE_ARN","arn:aws:iam::123456789012:role/SageMakerExecutionRole"),
    entry_point="inference.py",  # Provide script with input/output handling
    framework_version="1.2-1",
    py_version="py3",
    sagemaker_session=sm_sess
)

predictor = sk_model.deploy(
    initial_instance_count=1,
    instance_type="ml.t2.medium",
    endpoint_name="zap-logreg-dev"
)

# Example invocation (JSON payload)
# response = predictor.predict({"instances":[[0.1,0.2,0,1,0,1]]})
# print(response)